Install ckiptagger for word segmentation and opencc for s2t



In [ ]:
pip install -U ckiptagger[tfgpu,gdown]

In [ ]:
pip install opencc-python-reimplemented

import packages and mount gdrive to get access to our data.

In [3]:
#mount gdrive and import packages
import pandas as pd
import numpy as np
import re
from opencc import OpenCC
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Note that the ckiptagger 'ws' modules requires its pretrained model to segment the sentences, so we have to ensure that we've already obtained its pretrained weights. (If not, use `data_utils.download_data_gdown("./") ` to download the model)

For the purpose of saving time, i already upload those models in our gdrive.

In [4]:
#solving the environment with ckiptagger
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

Load the WS pretrained model (using gpu mode)

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#the WS module is alreday downloaded (in this project,I place it at my gdrive), if you havn't downloaded, use data utils.to download, also change the dir of WS
#data_utils.download_data_gdown("./") # gdrive-ckip
#ws = WS("./data")
ws=WS("/content/gdrive/My Drive/data", disable_cuda=False)

Here we use re to compile all the contents, and preserve only Chinese character.

Second, we transform all the Chinese characters from simplified Chinese to traditional Chinese.

And finally we find all the empty valuse and drop its column.

In [ ]:
review=pd.read_json('/content/gdrive/My Drive/Colab Notebooks/Dataset/Original_dataset/reviews.json')

cc = OpenCC('s2t')
for i in range(len(review)):

  tmp=review['content'][i].replace('\n', '')
  cop=re.compile("[^\u4e00-\u9fa5]")
  review.loc[review.content==review['content'][i],'content']=cop.sub('',tmp)
  review.loc[review.content==review['content'][i],'content']=cc.convert(review['content'][i])
  if review['content'][i].strip()=='':
    review=review.drop(index=i)


After dropping the columns, reindex the dataframe.

In [ ]:
review=review.reset_index(drop=True)

After segmenting the sentence, we can get each word in a sentence separately,


reconstruct them into a sentence using space to separate.

Here we also set a timer to measure if GPU acceleration is needed (in comparison with running on only CPU itself)



In [ ]:
import time
#to check if gpu is necessary
start_time = time.time()


#cut the corpus
tmp_corpus=ws(review['content'])
cut_corpus=[]
for j in tmp_corpus:
  cut_corpus.append(' '.join(j))

for k in range(len(review)):
  review.loc[review.content==review['content'][k],'content']=cut_corpus[k]

print("My program took",time.time() - start_time, "seconds to run on gpu")


My program took 5.933941602706909 seconds to run on gpu


See how well the ws performs.

In [ ]:
review['content'][0]

'播 歌 給 手機 聽 讓 它 找出 歌曲 來 很 好 雖然 都 可以 辨識出 歌曲 但 有些 歌曲 沒有 收錄 喔 原來 是 這樣 我 知道 了'

Compress the rating into [1,-1]

In [ ]:

rating_dict={1:-1,2:-0.5,3:0,4:0.5,5:1}
review.loc[review.rating==1, 'rating'] = rating_dict[1]
review.loc[review.rating==2, 'rating'] = rating_dict[2]
review.loc[review.rating==3, 'rating'] = rating_dict[3]
review.loc[review.rating==4, 'rating'] = rating_dict[4]
review.loc[review.rating==5, 'rating'] = rating_dict[5]

review.head()


,content,rating
0,播 歌 給 手機 聽 讓 它 找出 歌曲 來 很 好 雖然 都 可以 辨識出 歌曲 但 有些...,1.0
1,暢順 好聽,1.0
2,不錯,1.0
3,很 不錯 的 大部分 原唱 基本 都 有 還可以 免費 試聽 下載 良心 軟件 了,1.0
4,剛剛 安裝 就 要 問 祖先 十八 代 真是 夠 了 乾脆 解除 安裝 並且 把 它 廣泛 宣傳,-1.0


Now we get data preprocess done, save the processed data (either in .json or .csv is fine).

In [ ]:
review.to_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset/Output_dataset/reviews_w_compress_rating.csv',encoding='utf-8',index=False)